In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp
import sympy as sp
import itertools

#defining all 4 base strategies and all 16 possible combinations

def identity(x):
    return x

def swap(x):
    return 1-x

def reset0(x):
    return 0

def reset1(x):
    return 1

base_strategies = [identity, swap, reset0, reset1]
all_strategies = list(itertools.product(base_strategies, repeat=2))

In [2]:
#generating possible deterministic P_XY|ZU values for all given u
def generate_P_XY_giv_ZU(u):
    f_u, g_u = all_strategies[u]
    #print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    def inner_entry(z):
        inner = np.zeros((2,2), dtype=int)
        x = f_u(z)
        y = g_u(x)
        inner[y, x] = 1

        return inner

    outer_level = np.array([inner_entry(0), inner_entry(1)])
    
    return outer_level


#generating possible deterministic P_Y_do_X values for all given u
def generate_P_Y_do_XU(u):
    f_u, g_u = all_strategies[u]

    do_values = np.zeros((2,2), dtype=int)
    for x in range(2):
        y = g_u(x)
        do_values[y, x] = 1
            
    return do_values

In [3]:
# creating model, setting an MVar for weights (propabilities of u), and getting the average weighted sum for P(XY|Z) and P(Y do X)

m = gp.Model("m1")
w = m.addMVar(16, lb=0, name="P_u")
average_P_XY_giv_Z = sum([w[i]*generate_P_XY_giv_ZU(i) for i in range(16)])
average_P_Y_do_X = sum([w[i]*generate_P_Y_do_XU(i) for i in range(16)])
m.update()
#print(average_P_XY_giv_Z)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [4]:
m.remove(m.getConstrs())
m.addConstr(w.sum() == 1)

prob_dist = {
    (0, 0, 0): 0.0, 
    (0, 0, 1): 0.03571428571428571, 
    (0, 1, 0): 0.07142857142857142, 
    (0, 1, 1): 0.10714285714285714, 
    (1, 0, 0): 0.14285714285714285, 
    (1, 0, 1): 0.17857142857142858, 
    (1, 1, 0): 0.21428571428571427, 
    (1, 1, 1): 0.25}

cardX, cardY, cardZ = 2, 2, 2

def P_Z(z):
    return sum([prob_dist[(z, X, Y)] for X in range(cardX) for Y in range(cardY)])
def P_ZXY(z, x, y):
    return prob_dist[(z, x, y)]
def P_XY_given_Z(x,y,z):
    return P_ZXY(z,x,y)/P_Z(z)

for z,x,y in itertools.product(range(2), repeat=3):
    m.addConstr(average_P_XY_giv_Z[z,y,x] == P_XY_given_Z(x,y,z))

P_Y_do_X = m.addMVar(shape=(cardY, cardX), vtype=GRB.CONTINUOUS, name="y do x", lb=0, ub=1)
for x,y in itertools.product(range(2), repeat=2):
    m.addConstr(average_P_Y_do_X[y,x] == P_Y_do_X[y,x])

m.setObjective(P_Y_do_X[1, 0], GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 20 columns and 84 nonzeros
Model fingerprint: 0x358e2c82
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+00]
Presolve removed 5 rows and 8 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 45 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   8.939394e-01   0.000000e+00      0s
       4    8.1818182e-01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.181818182e-01


In [5]:
# def ACE(Y,X):
#     max(|P_Y_do_X[Y,X] - P_Y_do_X[Y,X]|, )